In [3]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout, Input, Concatenate
from sklearn.metrics import classification_report
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
from tensorflow.keras.callbacks import EarlyStopping

# Load the dataset
data = pd.read_csv("dataset.csv")

# Extract the image paths and additional features
image_paths = data["image_path"]
labels = data["artist_label"]
additional_features = data.drop(columns=["image_path", "artist_label"])

# Function to load and preprocess images
def load_and_preprocess_images(image_paths, target_size=(128, 128)):
    images = []
    for img_path in image_paths:
        img = image.load_img(img_path, target_size=target_size)
        img_array = image.img_to_array(img)
        images.append(img_array)
    return np.array(images)

# Load and preprocess images
X_images = load_and_preprocess_images(image_paths)
X_images = X_images / 255.0  # Normalize the images to [0, 1] range

# Convert labels to numpy array and one-hot encode them
y = np.array(labels)
y = to_categorical(y, num_classes=3)  # 3 classes (0, 1, 2)

# Convert additional features to numpy array
X_additional = additional_features.to_numpy()

# Normalize the additional features using StandardScaler
scaler = StandardScaler()
X_additional = scaler.fit_transform(X_additional)

# Split data into training and testing sets
X_train_images, X_test_images, X_train_additional, X_test_additional, y_train, y_test = train_test_split(
    X_images, X_additional, y, test_size=0.2, random_state=42
)

# Load the pre-trained VGG16 model without the top layers (classification part)
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

# Freeze the base model layers so that their weights are not updated during training
base_model.trainable = False

# Image input branch
image_input = Input(shape=(128, 128, 3), name='image_input')
x = base_model(image_input)
x = Flatten()(x)

# Additional features input branch
additional_input = Input(shape=(X_additional.shape[1],), name='additional_input')
y = Dense(64, activation='relu')(additional_input)

# Concatenate image features with the processed additional features
z = Concatenate()([x, y])

# Fully connected layers for classification
z = Dense(128, activation='relu')(z)
z = Dropout(0.5)(z)
output = Dense(3, activation='softmax')(z)  # 3 classes, softmax activation

# Define the complete model
model = Model(inputs=[image_input, additional_input], outputs=output)

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',  # Categorical cross-entropy for multi-class classification
              metrics=['accuracy'])

# Set up EarlyStopping
early_stopping = EarlyStopping(monitor='val_accuracy',  # Monitor validation accuracy
                               patience=3,  # Stop after 3 epochs without improvement
                               restore_best_weights=True,  # Restore the best weights when stopping
                               verbose=1)

# Train the model with EarlyStopping
history = model.fit(
    [X_train_images, X_train_additional],  # Pass both image and additional features for training
    y_train,
    epochs=10,
    batch_size=32,
    validation_data=([X_test_images, X_test_additional], y_test),  # Validation data
    callbacks=[early_stopping]  # Include EarlyStopping in the callbacks
)
# Evaluate the model
test_loss, test_acc = model.evaluate([X_test_images, X_test_additional], y_test)
print(f"Test accuracy: {test_acc}")

# Predict on the test set
y_pred = np.argmax(model.predict([X_test_images, X_test_additional]), axis=1)  # Convert softmax probabilities to class labels

# Get the actual class labels (as integers)
y_true = np.argmax(y_test, axis=1)

# Classification report for performance evaluation
report = classification_report(y_true, y_pred, target_names=['Class 0', 'Class 1', 'Class 2'])
print(report)


c:\Users\Chien Shyong\Documents\School Work\Term 8\Data Science\yukien\myvenv\Lib\site-packages\PIL\Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch 1/10
61/61 ━━━━━━━━━━━━━━━━━━━━ 46s 732ms/step - accuracy: 0.6663 - loss: 0.7939 - val_accuracy: 0.8932 - val_loss: 0.3196
Epoch 2/10
61/61 ━━━━━━━━━━━━━━━━━━━━ 43s 714ms/step - accuracy: 0.9021 - loss: 0.2764 - val_accuracy: 0.9302 - val_loss: 0.2250
Epoch 3/10
61/61 ━━━━━━━━━━━━━━━━━━━━ 45s 747ms/step - accuracy: 0.9293 - loss: 0.2069 - val_accuracy: 0.9425 - val_loss: 0.1954
Epoch 4/10
61/61 ━━━━━━━━━━━━━━━━━━━━ 45s 746ms/step - accuracy: 0.9495 - loss: 0.1407 - val_accuracy: 0.9569 - val_loss: 0.1745
Epoch 5/10
61/61 ━━━━━━━━━━━━━━━━━━━━ 43s 706ms/step - accuracy: 0.9601 - loss: 0.1238 - val_accuracy: 0.9487 - val_loss: 0.1713
Epoch 6/10
61/61 ━━━━━━━━━━━━━━━━━━━━ 43s 705ms/step - accuracy: 0.9781 - loss: 0.0761 - val_accuracy: 0.9384 - val_loss: 0.2102
Epoch 7/10
61/61 ━━━━━━━━━━━━━━━━━━━━ 43s 714ms/step - accuracy: 0.9756 - loss: 0.0670 - val_accuracy: 0.9405 - val_loss: 0.2261
Epoch 7: early stopping
Restoring model weights from the end of the best epoch: 4.
16/16 ━━━━━━━━